# Ethiopia Conflict Viewer

This notebook is an experimentation with [folium](https://github.com/python-visualization/folium) for geospatial data visualization in Python.  It creates a dynamic visualization of [ACLED](https://www.acleddata.com/) conflict data for Ethiopia from 1997 - present day.  The ACLED Database chronicles incidents of violence in the Middle East and Africa.  It is manually updated each week and includes details about each incident, including the location, the actors involved, the type of conflict (one of 9 classifications), the number of fatalities, and the original news source that reported the event.  

I followed [python_cartography_tutorial](https://github.com/vincentropy/python_cartography_tutorial/blob/master/part3_animations.ipynb) for the folium plotting in this notebook.  Install the latest version of folium from python:

`pip install --upgrade git+https://github.com/python-visualization/folium`

In [2]:
# %matplotlib inline
import os
import bisect
import branca
import datetime
import geopandas as gpd
import imageio
import numpy as np
import pandas as pd
import requests
import time
import urllib

from PIL import Image, ImageDraw, ImageFont
from selenium import webdriver

import folium
from folium import plugins
from folium.features import DivIcon

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

from bs4 import BeautifulSoup
from dateutil import rrule
from IPython.display import display, HTML
from shapely.geometry import Point

# Establishing plotting settings 
pd.set_option('display.max_colwidth', -1)
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Tahoma']
plt.rcParams['figure.dpi'] = 200
%config InlineBackend.figure_format = 'svg'

HOME_DIR = os.getcwd()

#### Scrape the ACLED Data

In [3]:
url = "https://www.acleddata.com/curated-data-files/"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
download = soup.find_all("a", class_="download-button")[0]
data_url = download.get("href")
data_filepath = os.path.join(HOME_DIR, "data")
os.makedirs(data_filepath, exist_ok=True)
data_download = urllib.request.urlretrieve(data_url, os.path.join(data_filepath, "acled_data.xlsx"))

#### Convert ACLED data to a shapefile and isolate incidents in Ethiopia

In [4]:
acled_data = pd.read_excel(os.path.join(data_filepath, "acled_data.xlsx"))
df_eth = acled_data[acled_data['COUNTRY'] == 'Ethiopia'].copy()
df_eth['geometry'] = df_eth.apply(lambda x: Point((float(x.LONGITUDE), float(x.LATITUDE))), axis=1)
df_eth = gpd.GeoDataFrame(df_eth, geometry='geometry').reset_index(drop=True)

# Show the first 5 records of the ACLED Data
display(pd.DataFrame(df_eth.loc[0:5]).T)

,0,1,2,3,4,5
ISO,231,231,231,231,231,231
EVENT_ID_CNTY,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6
EVENT_ID_NO_CNTY,1,2,3,4,5,6
EVENT_DATE,1997-01-15 00:00:00,1997-02-10 00:00:00,1997-02-15 00:00:00,1997-03-21 00:00:00,1997-03-30 00:00:00,1997-04-12 00:00:00
YEAR,1997,1997,1997,1997,1997,1997
TIME_PRECISION,3,1,3,1,1,1
EVENT_TYPE,Strategic development,Violence against civilians,Headquarters or base established,Violence against civilians,Riots/Protests,Remote violence
ACTOR1,SPLA/M: Sudanese Peoples Liberation Army/Movement,Unidentified Armed Group (Ethiopia),SPLA/M: Sudanese Peoples Liberation Army/Movement,Police Forces of Ethiopia (1991-),Protesters (Ethiopia),Unidentified Armed Group (Ethiopia)
ASSOC_ACTOR_1,NaN,NaN,NaN,NaN,NaN,NaN
INTER1,2,3,2,1,6,3


#### Read in the admin boundaries for Ethiopia to extract the center point to tell the map where to center.

In [5]:
admin_df = gpd.read_file(os.path.join(data_filepath, 'ethiopia/ethiopia.shp'))
admin0_df = admin_df.dissolve(by='COUNTRY').reset_index()
poly = gpd.GeoSeries(admin0_df.loc[0,'geometry'])
center_loc = [poly.representative_point()[0].y, poly.representative_point()[0].x]
month_list = [dt for dt in rrule.rrule(rrule.MONTHLY,
                                       dtstart=datetime.datetime(1998, 1, 1, 0, 0),
                                       until=datetime.datetime.today())]

#### Define functions for data extraction and plotting
`get_incidents_by_month` simply extracts all of the conflict incidents for the month that is passed.  This allows you to select the month you want to view and plot. `generate_map` creates a map with all of the conflict incidents from the selected month.  Each point will be scaled based on the number of fatalities that occurred during the incident.  Each point will also have details on the incident that it represents, including the exact date and a description of the conflict.  Any data contained in the ACLED data file could be added as descriptive metadata in the pop-up bubble if desired.

In [6]:
def get_incidents_by_month(df, month, month_list=month_list, geojson_flg=False):
    ind = month_list.index(month)
    date = month.__str__().split(' ')[0]+'T'+month.__str__().split(' ')[1]
    df_month = df.loc[(df['EVENT_DATE'] > month_list[ind]) &
                      (df['EVENT_DATE'] < month_list[ind + 1])]
    if geojson_flg is True:
        try:
            geojson_month = df_month.__geo_interface__
            geojson_month['features'][0]['properties']['dates'] = date
        except ValueError:
            geojson_month = {'features': {'dates' : date}}
        return geojson_month['features']
    else:
        return df_month

def generate_map(df, month, zoom_start=6, center_loc=center_loc):
    m = folium.Map(location=center_loc,
                            zoom_start=5,
                            tiles="Mapbox Control Room",
                            width="100%")

    # for each row in the data, add a cicle marker
    for index, row in df.iterrows():

        num_fatalities = row["FATALITIES"]

        popup_text = """
            <h4> {} </h4>
            <p><i> {} </i></p>
            <p> {} </p>
            <p style="color:#af0f00"><i> {} fatalities </i></p>
            """
        notes = row['NOTES']
        if type(notes) is float:
            notes = ''
        elif ':' in notes:
            notes = notes.split(':')[1]
        date = row['EVENT_DATE'].to_pydatetime().strftime('%d %b, %Y')
        popup_text = popup_text.format(row['EVENT_TYPE'], date, notes, row['FATALITIES'])

        iframe = branca.element.IFrame(html=popup_text, width=250, height=200)
        popup = folium.Popup(iframe, max_width=250)

        # radius of circles
        bins = [0, 1, 5, 10, 20, 30, 50, 75, 100, 150, 200, 500, 1000, 2000]
        radius =  (bisect.bisect(bins, num_fatalities)**2)/1.5
        if radius == 0:
            radius = 3
        
        if row['EVENT_TYPE'] == 'Violence against civilians':
            color = '#fc3535'
        elif row['EVENT_TYPE'] == 'Riots/Protests':
            color = '#b903bf'
        else:
            color="#f79f25"

        # add marker to the map
        folium.CircleMarker(location=(row["LATITUDE"],
                                      row["LONGITUDE"]),
                            radius=radius,
                            color=color,
                            popup=popup,
                            fill=True).add_to(m)
    return m

#### Pick your month and Create the Map
Type the month you want to show in the `month` variable.

The markers' size scales with the number of fatalities while the color indicates the event types.  I was particularly interested in highlighting violence involving civilians, so the event types are classified as follows:
 - <font color='#fc3535'>Red</font> = Violence against civilians
 - <font color='#b903bf'>Purple</font> = Riot or Protest
 - <font color='#f79f25'>Orange</font> = All other event types

In [7]:
# The format should be "Feb 2019".
month = 'Feb 2019'

#  Plot the map
if type(month) is str:
    month = datetime.datetime.strptime(month, '%b %Y')
df = get_incidents_by_month(df_eth, month)
m = generate_map(df, month)
m

## Part 2: Create a GIF
There is no good method to save png or pdf files from folium yet, so we use this workaround of using selenium to open the html in Firefox and take a screenshot.  For this step, you must have downloaded [geckodriver](https://github.com/mozilla/geckodriver/releases/) and placed the executable file in your base directory.

In [12]:
gif_filepath = os.path.join(HOME_DIR, 'gif2')
os.makedirs(gif_filepath, exist_ok=True)
browser = webdriver.Firefox(executable_path=r'{}/geckodriver'.format(HOME_DIR))

for month in month_list[150: -1]:
    df = get_incidents_by_month(df_eth, month)
    m = generate_map(df, month)
    delay=5
    fn='tempmap.html'
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    m.save(fn)
    browser.get(tmpurl)

    #Give the map tiles some time to load
    time.sleep(delay)
    png = os.path.join(gif_filepath, '{}.png'.format(month.strftime('%b_%Y')))
    browser.save_screenshot(png)

    # create a PIL image object
    image = Image.open(png)
    draw = ImageDraw.ImageDraw(image)
    font = ImageFont.truetype(fm.findfont(fm.FontProperties(family='Verdana')), 40)
    
    # draw title
    draw.text((image.width - 400, 20), 
              "{}".format(month.strftime("%b %Y")),
              fill="#f79f25",
             font=font)

    image.save(png, "PNG", optimize=True, quality=95)
    
browser.quit()

#### Take all the saved images and stitch them into an html

In [13]:
images = []
files = os.listdir(gif_filepath)
files.sort(key=lambda x: os.path.getmtime(os.path.join(gif_filepath, x)))

for fn in files:
    if fn.endswith('.png'):
        images.append(imageio.imread(os.path.join(gif_filepath, fn)))
imageio.mimsave(os.path.join(HOME_DIR, 'conflict_ethiopia.gif'), images, fps=0.5)

In [ ]:
with open(os.path.join(HOME_DIR, 'conflict_ethiopia.gif'), 'rb') as f:
    display(IPython.display.Image(data=f.read(), format='png', width=800, height=800))